In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tssb.utils import load_time_series_segmentation_datasets
from tssb.evaluation import covering

In [ ]:
# loading datasets
tssb = load_time_series_segmentation_datasets(names=["ArrowHead", "InsectWingbeatSound", "TwoLeadECG"])

In [ ]:
tssb

In [ ]:
line_ds = tssb.loc[tssb['dataset'] == 'InsectWingbeatSound']
line_ts = line_ds['time_series'].values[0]
line_chp = line_ds['change_points'].values[0]
line_chp

In [ ]:
plt.figure(figsize=(8,3))
cur_idx = 0
for i in range(len(line_chp)):
    plt.plot(np.arange(cur_idx,line_chp[i]), line_ts[cur_idx:line_chp[i]])
    cur_idx = line_chp[i]
plt.plot(np.arange(cur_idx,len(line_ts)-1), line_ts[cur_idx:len(line_ts)-1])
plt.show()

In [ ]:
from CLaSP import CLaSP

In [ ]:
cls = CLaSP(window=10, solver='knn')

In [ ]:
cls.fit(line_ts, n_segments=4)

In [ ]:
cls.stack

In [ ]:
plt.imshow(cls.X_dist, cmap='seismic')
plt.show()